Playing musics from models

In [1]:
from keras.models import Model, load_model
import numpy as np
import utils as ut
import PreProcessor as pp
from mido import Message, MetaMessage, MidiFile, MidiTrack, bpm2tempo, second2tick

In [2]:
seq_length = 30

#dataset = pp.load_dataset("../adl-piano-midi")
dataset = pp.load_dataset("C:/Users/mxrin/OneDrive/Documents/KTH_2/DD2424/Projet/DD2424-MusicGen/adl-piano-midi")
ClassicSongs = pp.files_to_songs(dataset["Classical"])

channel_to_ind, ind_to_channel, note_to_ind, ind_to_note, velocity_to_ind, ind_to_velocity = pp.dicts_from_songs(ClassicSongs,velocities=True)
time_range = pp.ranges_from_songs(ClassicSongs)

n_Channels, n_Notes, n_Velocities = len(channel_to_ind), len(note_to_ind), len(velocity_to_ind)

print("\nNumber of channels   :",n_Channels,"\nNumber of notes      :",n_Notes,"\nNumber of velocities :",n_Velocities,"\nTime range           :",time_range[0],time_range[1])

Couldn't load : C:/Users/mxrin/OneDrive/Documents/KTH_2/DD2424/Projet/DD2424-MusicGen/adl-piano-midi\adl-piano-midi\Classical\Classical\Alexander Borodin\Nocturne.mid
Couldn't load : C:/Users/mxrin/OneDrive/Documents/KTH_2/DD2424/Projet/DD2424-MusicGen/adl-piano-midi\adl-piano-midi\Classical\Classical\Antonio Salieri\Invention.mid
Couldn't load : C:/Users/mxrin/OneDrive/Documents/KTH_2/DD2424/Projet/DD2424-MusicGen/adl-piano-midi\adl-piano-midi\Classical\Classical\Bela Bartok\Andante.mid
Couldn't load : C:/Users/mxrin/OneDrive/Documents/KTH_2/DD2424/Projet/DD2424-MusicGen/adl-piano-midi\adl-piano-midi\Classical\Classical\Bela Bartok\Romanian Folk Dance.mid
Couldn't load : C:/Users/mxrin/OneDrive/Documents/KTH_2/DD2424/Projet/DD2424-MusicGen/adl-piano-midi\adl-piano-midi\Classical\Classical\Claude Debussy\Clair De Lune.mid
Couldn't load : C:/Users/mxrin/OneDrive/Documents/KTH_2/DD2424/Projet/DD2424-MusicGen/adl-piano-midi\adl-piano-midi\Classical\Classical\Franz Schubert\Impromptu in C-

In [3]:
LSTM_model = ut.create_model("LSTM",250,n_Channels,n_Notes,n_Velocities,time_range)
LSTM_model.load_weights("LSTM_model.h5")

In [7]:
seq_length = 30

In [8]:
X_Channels, X_Notes, X_Velocities, X_Times, y_Channels, y_Notes, y_Velocities, y_Times, Val_X_Channels, Val_X_Notes, Val_X_Velocities, Val_X_Times, Val_y_Channels, Val_y_Notes, Val_y_Velocities, Val_y_Times, Test_X_Channels, Test_X_Notes, Test_X_Velocities, Test_X_Times, Test_y_Channels, Test_y_Notes, Test_y_Velocities, Test_y_Times = ut.prep_data(seq_length, ClassicSongs,n_Channels,n_Notes,channel_to_ind,note_to_ind)

In [10]:
gen = ut.generate(LSTM_model,[X_Channels[2:3,:,:], X_Notes[2:3,:,:], X_Velocities[2:3,:,:], X_Times[2:3,:,:]],30,ind_to_channel,ind_to_note,mode="SoftMax",T=0.8)
gen

ValueError: in user code:

    File "C:\Users\mxrin\AppData\Roaming\Python\Python311\site-packages\keras\engine\training.py", line 2169, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\mxrin\AppData\Roaming\Python\Python311\site-packages\keras\engine\training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\mxrin\AppData\Roaming\Python\Python311\site-packages\keras\engine\training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\mxrin\AppData\Roaming\Python\Python311\site-packages\keras\engine\training.py", line 2111, in predict_step
        return self(x, training=False)
    File "C:\Users\mxrin\AppData\Roaming\Python\Python311\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\mxrin\AppData\Roaming\Python\Python311\site-packages\k

In [34]:
channels,notes,velocities,times = ut.create_midi(gen)
print(channels)
time_ = [int(l*960) for l in times]
print(notes)
print(velocities)
print(times)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
[64, 69, 69, 69, 69, 81, 69, 64, 60, 62, 62, 65, 67, 62, 62, 62, 62, 62, 62, 69, 69, 63, 69, 69, 64, 61, 64, 62, 62, 66]
[49, 26, 16, 11, 8, 7, 5, 7, 9, 10, 8, 8, 10, 10, 8, 8, 8, 8, 8, 8, 8, 8, 10, 8, 7, 9, 11, 10, 8, 7]
[0.073918596, 0.47434297, 0.8796422, 0.9951967, 1.0318245, 1.0809896, 1.2220612, 1.2423832, 0.92080295, 1.0990905, 1.3137932, 1.3523624, 0.96484005, 0.9040212, 1.2957172, 1.3505639, 1.3558822, 1.3563778, 1.3564228, 1.3564291, 1.1705244, 1.1168983, 1.0696937, 1.0564992, 1.0870595, 0.8534793, 1.1653556, 0.8853866, 1.2934879, 2.4373732]


In [30]:
mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)

track.append(MetaMessage('key_signature', key='Dm'))
#track.append(MetaMessage('set_tempo', tempo=bpm2tempo(960)))
track.append(MetaMessage('time_signature', numerator=6, denominator=8))

for i in range(len(channels)):
    track.append(Message('note_on', channel=channels[i], note=notes[i], velocity=velocities[i], time=time_[i]))
track.append(MetaMessage('end_of_track'))
mid.save('new_song.mid')

In [31]:
import pygame
pygame.init()
pygame.mixer.music.load("new_song.mid")
pygame.mixer.music.play()